### 1. function generateSPDmatrix

In [1]:
using LinearAlgebra

In [2]:
function generateSPDmatrix(n)
    A = rand(n,n)
    B = Symmetric(A,:U)
    B = B + n*Matrix(I(n))
    return B
end

generateSPDmatrix (generic function with 1 method)

### 2. function choleskyDecompose

In [3]:
function choleskyDecompose!(A::Array{Float64,2})
    n = length(A[:,1])
    
    for k in 1:n
        if A[k,k] < 0
            error("Matrix is not positive definite")
        end
        A[k,k] = sqrt(A[k,k] - dot(A[k,1:k-1],A[k,1:k-1]))
        for i in k+1:n
            A[i,k] = (A[i,k] - dot(A[i,1:k-1],A[k,1:k-1])) / A[k,k]
        end
    end
    
    for k in 2:n
        A[1:k-1,k] = zeros(k-1)
    end
    
    return A
end
    

choleskyDecompose! (generic function with 1 method)

### 3. function choleskySolve

In [4]:
function choleskySolve!(L::Array{Float64,2},b::Array{Float64,1})
    n = length(b)
    
    #Solution of [L]{y} = {b}
    for k in 1:n
        b[k] = (b[k] - dot(L[k,1:k-1],b[1:k-1]))/L[k,k]
    end
    
    # Solution of [L_transpose]{x} = {y}
    for k in n:-1:1
        b[k] = (b[k] - dot(L[k+1:n,k],b[k+1:n])) /L[k,k]
    end
    return b
end


choleskySolve! (generic function with 1 method)

In [5]:
A = generateSPDmatrix(100)

100×100 Array{Float64,2}:
 100.115        0.8732       0.597079   …    0.508455     0.695653 
   0.8732     100.502        0.787069        0.541899     0.773271 
   0.597079     0.787069   100.282           0.610013     0.209749 
   0.472819     0.787806     0.911917        0.480584     0.403177 
   0.490181     0.830139     0.651598        0.708868     0.815973 
   0.203956     0.149089     0.339931   …    0.432961     0.699816 
   0.843853     0.0652933    0.493653        0.385803     0.0173841
   0.605863     0.142445     0.957244        0.224763     0.124287 
   0.713121     0.660057     0.664708        0.0293402    0.341878 
   0.479154     0.840888     0.357113        0.178227     0.320699 
   0.94325      0.715489     0.938002   …    0.846554     0.457177 
   0.895344     0.195701     0.127699        0.135896     0.071945 
   0.857438     0.245542     0.0863232       0.624115     0.290725 
   ⋮                                    ⋱                          
   0.0276325    0.2363

In [6]:
b = rand(100)

100-element Array{Float64,1}:
 0.8858662323215323 
 0.9798807727259333 
 0.24007073708353732
 0.36670358841821105
 0.01960279494106021
 0.36481423118648193
 0.14881380842324288
 0.42582314896766604
 0.6773953345627166 
 0.5913858194001005 
 0.3666753693222573 
 0.21629926181779568
 0.830966168766432  
 ⋮                  
 0.1497751401955436 
 0.6889364722396787 
 0.0511534144888941 
 0.733353189374133  
 0.22184867414852238
 0.47415302137489235
 0.14225760682491573
 0.44581556922759025
 0.14431245236674628
 0.6233628680754508 
 0.04643450214879197
 0.2759699612875739 

### Julia’s \ functionality

In [7]:
@timev A \ b

  1.050690 seconds (3.87 M allocations: 187.066 MiB, 5.94% gc time)
elapsed time (ns): 1050689688
gc time (ns):      62400391
bytes allocated:   196152628
pool allocs:       3872153
non-pool GC allocs:726
malloc() calls:    2
GC pauses:         9


100-element Array{Float64,1}:
  0.0072312313335380195 
  0.00815987989365772   
  0.0008238835598002542 
  0.0021608915002543534 
 -0.0014123076384940682 
  0.001955757583733038  
 -0.00025646467278897994
  0.0026311909677279493 
  0.005156813894773365  
  0.004523896566973614  
  0.0021209063030110625 
  0.00023432929617030342
  0.0063858002839664325 
  ⋮                     
 -3.304742116164591e-6  
  0.005159891828522059  
 -0.0009351402367257511 
  0.005526314916381282  
  0.0004927464157269204 
  0.0030625542654638444 
 -3.100535336431014e-5  
  0.002784970018289827  
 -8.11880851695054e-5   
  0.004655298735666107  
 -0.001073294127940109  
  0.0010727832159067097 

In [8]:
@timev L = choleskyDecompose!(A)

  0.125587 seconds (270.97 k allocations: 16.250 MiB)
elapsed time (ns): 125586720
bytes allocated:   17038849
pool allocs:       270928
non-pool GC allocs:46


100×100 Array{Float64,2}:
 10.0057       0.0          0.0         …   0.0          0.0         0.0   
  0.0872699   10.0247       0.0             0.0          0.0         0.0   
  0.0596736    0.0779936   10.0136          0.0          0.0         0.0   
  0.0472548    0.0781753    0.0901772       0.0          0.0         0.0   
  0.04899      0.082383     0.0641375       0.0          0.0         0.0   
  0.0203839    0.0146947    0.0337109   …   0.0          0.0         0.0   
  0.0843369    0.00577906   0.0487505       0.0          0.0         0.0   
  0.0605515    0.0136823    0.0951267       0.0          0.0         0.0   
  0.0712712    0.0652228    0.0654476       0.0          0.0         0.0   
  0.047888     0.0834649    0.0347273       0.0          0.0         0.0   
  0.0942709    0.070552     0.0925612   …   0.0          0.0         0.0   
  0.0894831    0.0187429    0.0120732       0.0          0.0         0.0   
  0.0856946    0.0237477    0.00792494      0.0          0.0  

In [9]:
@timev choleskySolve!(L,b)

  0.122974 seconds (152.34 k allocations: 7.621 MiB, 17.01% gc time)
elapsed time (ns): 122973771
gc time (ns):      20922587
bytes allocated:   7991487
pool allocs:       152306
non-pool GC allocs:36
GC pauses:         1
full collections:  1


100-element Array{Float64,1}:
  0.007231231333538017  
  0.00815987989365772   
  0.0008238835598002541 
  0.0021608915002543543 
 -0.0014123076384940677 
  0.0019557575837330396 
 -0.00025646467278898   
  0.0026311909677279493 
  0.0051568138947733645 
  0.004523896566973616  
  0.0021209063030110603 
  0.00023432929617030356
  0.006385800283966431  
  ⋮                     
 -3.3047421161645177e-6 
  0.005159891828522059  
 -0.000935140236725752  
  0.005526314916381285  
  0.0004927464157269206 
  0.0030625542654638452 
 -3.100535336430986e-5  
  0.002784970018289828  
 -8.118808516950563e-5  
  0.004655298735666108  
 -0.0010732941279401084 
  0.0010727832159067095 

### Concusion: The last digits of the two method answers are not the same because of the computer float point property. But the answers are both correct.